In [ ]:
from proximal_operators import *
from auxiliary_functions import *
from algos import *
from test import *

In [ ]:
dim = 10000  # add checks for correct accuracies...
seed = None

# Dykstra

In [ ]:
projs = [
    partial(proj_box, lwb=0, upb=0.5),
    partial(proj_affine_hyperplane, vec_or=np.ones(dim), intercept=1)]

In [ ]:
%%timeit
sol, errs, vrbls = dykstra(test_gen_mat_vec(seed, dim)[1], projs)
# print(vrbls.keys())
# fit_log_errs(errs, True)

# Long-only fully invested

In [ ]:
%%timeit
sol_apgd_simplex, errs, vrbls = apgd(*test_gen_mat_vec(seed, dim), proj_simplex, som=1)
# print(vrbls.keys())
# fit_log_errs(errs, True)

# Long-only fully invested with box constraints

In [ ]:
projs = [
    partial(proj_box, lwb=0, upb=0.1),
    partial(proj_affine_hyperplane, vec_or=np.ones(dim), intercept=1)]
proj = lambda vec: dykstra(vec, projs)[0]

In [ ]:
%%timeit
sol_apgd_simplex, errs, vrbls = apgd(*test_gen_mat_vec(seed, dim), proj)
# print(vrbls.keys())
# fit_log_errs(errs, True)

In [ ]:
# Expes

DIM = 1000
orig = np.random.randn(DIM, 10 * DIM)
CST = 1
matrix = CST * np.matmul(orig, orig.T)
vector = np.random.randn(DIM)
metric = partial(norm, ord=float('inf'))
errs_dic = dict()

## Expe 1

constraints = [
    partial(proj_box, lwb=-0.5, upb=0.5),
    partial(proj_l1_ball, rad=1),
    partial(proj_affine_hyperplane, vec_or=np.ones(DIM), intercept=0)
]

%%time
func = prox_grad
result, errors = func(matrix, vector, metric, constraints, {'erg': 1e-6})
pd.Series(result).hist(bins=100)
errs_dic[func.__name__] = errors
[metric(constraint(result) - result) for constraint in constraints]

%%time
func = loris_verhoeven
result, errors = func(matrix, vector, metric, constraints, {'erg': 1e-6})
pd.Series(result).hist(bins=100)
errs_dic[func.__name__] = errors
[metric(constraint(result) - result) for constraint in constraints]

errs_frame = pd.concat([
    errs.rename(columns=dict(zip(errs.columns, name + errs.columns)))
    for name, errs in errs_dic.items()], axis=1)
errs_frame.plot(grid=True, logy=True, figsize=(20, 15))

## Expe 2

constraints = [
    partial(proj_box, lwb=0, upb=0.1),
    partial(proj_simplex_quick, rad=1),
]

%%time
func = prox_grad
result, errors = func(matrix, vector, metric, constraints, {'erg': 1e-6})
pd.Series(result).hist(bins=100)
errs_dic[func.__name__] = errors
[metric(constraint(result) - result) for constraint in constraints]

%%time
func = loris_verhoeven
result, errors = func(matrix, vector, metric, constraints, {'erg': 1e-6})
pd.Series(result).hist(bins=100)
errs_dic[func.__name__] = errors
[metric(constraint(result) - result) for constraint in constraints]

errs_frame = pd.concat([
    errs.rename(columns=dict(zip(errs.columns, name + errs.columns)))
    for name, errs in errs_dic.items()], axis=1)
errs_frame.plot(grid=True, logy=True, figsize=(20, 15))